In [21]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt

from deepface import DeepFace
import math

# Setting

In [22]:
metrics = ["cosine", "euclidean", "euclidean_l2"] # 유사도 설정 가능

workpath = "C:\\Users\\ksko\\Desktop\\work\\"
filepath = "C:\\Users\\ksko\\Desktop\\work\\m_deep\\"


src = "C:\\Users\\ksko\\Desktop\\work\\m_00003.jpg"

total_data = os.listdir(filepath)

filepath 에는 동영상이 들어있는 폴더 경로
workpath 에는 filepath의 전단계 경로

src에는 src로 비교할 이미지 한개 경로

최종적으로 total_data에 들어가는 경로가 동영상이 전부 들어있는 폴더입니다.

In [23]:
total_data[:3]

['관계01_Mission_Complete_Reaction.mp4',
 '관계01_Mission_Start_Description_En.mp4',
 '관계01_Mission_Start_Guide_En.mp4']

In [24]:
scenario = ['인사', '시험공부01','시험공부02','면접01','사랑01','전화01','비지니스01','대화와응답01',
            '의견파악01','관계01','할일','숫자정보01','숫자정보02']
# 숫자정보01 : num1
# 숫자정보02 : num2
# 할일,부탁할일01 : todo

동영상 맨 앞 제목을 scenario list에 적어서 넣어주시면됩니다.

저의 경우 숫자정보01이 num1_~~~ 로 저장되어 있어 해당부분을 수정하여 넣어둔 상태입니다.

그러고 나서 아래 코드도 이름 맞춰 넣어주시면 됩니다.

In [25]:
# 영상 제목들을 넣을 리스트
greet = []
test01 = []
test02 = []
interview = []
love = []
call = []
business = []
conversation = []
opinion = []
relation = []
todo = []
num1 = []
num2 = []

# 영상 제목들을 추가
for i in range(len(total_data)):
    if total_data[i][:2] == '인사':
        greet.append(total_data[i])
    if total_data[i][:6] == '시험공부01':
        test01.append(total_data[i])
    elif total_data[i][:6] == '시험공부02':
        test02.append(total_data[i])
    elif total_data[i][:2] == '면접':
        interview.append(total_data[i])
    elif total_data[i][:2] == '사랑':
        love.append(total_data[i])
    elif total_data[i][:2] == '전화':
        call.append(total_data[i])
    elif total_data[i][:2] == '비지':
        business.append(total_data[i])
    elif total_data[i][:2] == '대화':
        conversation.append(total_data[i])
    elif total_data[i][:2] == '의견':
        opinion.append(total_data[i])
    elif total_data[i][:2] == '관계':
        relation.append(total_data[i])
    elif total_data[i][:2] == '할일':
        todo.append(total_data[i])
    elif total_data[i][:6] == '숫자정보01':
        num1.append(total_data[i])
    elif total_data[i][:6] == '숫자정보02':
        num2.append(total_data[i])
#    if total_data[i][:6] == '인사': # 인사가 빠짐!!
#        test01.append(total_data[i])


# 영상의 유사도를 저장하는 리스트
greet_result = []
test01_result = []
test02_result = []
interview_result = []
love_result = []
call_result = []
business_result = []
conversation_result = []
opinion_result = []
relation_result = []
todo_result = []
num1_result = []
num2_result = []


# 영상 제목을 넣는 리스트를 묶은 리스트 (이중리스트)
name_list = [greet, test01, test02, interview, love, 
            call, business, conversation, opinion, 
            relation, todo, num1, num2]

# 영상의 유사도를 저장하는 리스트를 묶은 리스트 (이중리스트)
result_list = [greet_result, test01_result, test02_result, interview_result, love_result,
              call_result, business_result, conversation_result, opinion_result,
              relation_result, todo_result, num1_result, num2_result]

In [26]:
list(map(len, (greet, test01, test02, interview, love, call, business, conversation, opinion, relation, todo, num1, num2)))
# 빈거 체크

[10, 12, 12, 12, 12, 12, 11, 10, 12, 14, 10, 14, 12]

# Code

In [28]:
# 시나리오별로 반복
#for i in range(len(scenario)):
for i in range(0, 1):
    print(f"--- 현재 진행중인 시나리오--- : {scenario[i]}")
    temp_name_list = name_list[i]
    temp_result_list = result_list[i]
    
    # 영상별로 반복
    for name in temp_name_list:
        path = filepath + name
        video = cv2.VideoCapture(path)
        if(video.isOpened()):
            print(f"현재 진행중인 영상 : {name}")
        length = video.get(cv2.CAP_PROP_FRAME_COUNT)
        cut = math.floor(length/10) # 구간을 10개로 나눈다
        if cut == 0: # 프레임이 1인경우 0으로 나누게 되어 오류가 남. 제외시키자
            continue
        count = 0
        
        print('영상의 프레임수', length)
        
        # 영상의 프레임별로 반복
        while(video.isOpened()):
            ret, img = video.read() # 다음 프레임 읽기
            
            if (int(video.get(1))%cut==0 & count <= 10): # 10개의 구간
                try:
                    result = DeepFace.verify(img1_path = img, # 유사도 측정
                              img2_path = src,
                              distance_metric = metrics[0],
                    )
                except: # 얼굴 인식이 안된경우 예외처리
                    count -= 1
                    pass
                count += 1                
                print(f'유사도 : {result["distance"]}\r', end='') # 유사도 출력
                # 11번째로 넘어가는 경우 추가하지 않는다.
                if count <= 10:
                    temp_result_list.append(result['distance'])
                else: count -= 1
                
            if video.get(1) >= length : 
                print("                              ", end='')
                print(f"\r나눠진 사진 수 : {count} \n")
                break
    # 완료된 시나리오에 대해 외부로 csv export            
    pd.Series(result_list[i]).to_csv(workpath + 'deep_m_' + scenario[i] + '.csv', index=False, header=False)

--- 현재 진행중인 시나리오--- : 인사
현재 진행중인 영상 : 인사_Mission_Complete_reaction.mp4
영상의 프레임수 121.0
1/1 [==============================] - 0s 155ms/step
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_Mission_Start_Description_En.mp4
영상의 프레임수 169.0
1/1 [==============================] - 0s 200ms/step
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_Mission_Start_Guide_en.mp4
영상의 프레임수 99.0
1/1 [==============================] - 0s 198ms/step
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_Task_1_Q3.mp4
영상의 프레임수 69.0
1/1 [==============================] - 0s 191ms/step
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_Task_1_Reaction_Reaction.mp4
영상의 프레임수 86.0
1/1 [==============================] - 0s 195ms/step
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_Task_2_Q1.mp4
영상의 프레임수 60.0
1/1 [==============================] - 0s 200ms/step
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_Task_2_Reaction_Reaction.mp4
영상의 프레임수 45.0
1/1 [==============================] - 0s 205ms/step
나눠진 사진 수 : 10           

In [58]:
for i in range(len(scenario)):
    print(f"{scenario[i]}의 유사도 평균은 {round(pd.Series(result_list[i]).mean(),2)}, 표준편차는 {round(pd.Series(result_list[i]).std(),2)} 입니다.")

시험공부01의 유사도 평균은 0.38, 표준편차는 0.15 입니다.
시험공부02의 유사도 평균은 0.33, 표준편차는 0.25 입니다.
면접01의 유사도 평균은 0.32, 표준편차는 0.25 입니다.
사랑01의 유사도 평균은 0.33, 표준편차는 0.25 입니다.
전화01의 유사도 평균은 0.32, 표준편차는 0.26 입니다.
비지니스01의 유사도 평균은 0.33, 표준편차는 0.25 입니다.
대화와응답01의 유사도 평균은 0.17, 표준편차는 0.19 입니다.
의견파악01의 유사도 평균은 0.34, 표준편차는 0.26 입니다.
관계01의 유사도 평균은 0.29, 표준편차는 0.24 입니다.
todo의 유사도 평균은 0.34, 표준편차는 0.25 입니다.
num1의 유사도 평균은 0.34, 표준편차는 0.25 입니다.
num2의 유사도 평균은 0.28, 표준편차는 0.25 입니다.
